In [10]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Exploring parameters settings using GridSearchCV

In [12]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [14]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5] # print out all the results across all the parameter settings

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are acc

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
7,35.184124,1.266797,0.551923,0.048772,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979372,0.976640,0.975741,...,0.973954,0.004224,1,0.999551,0.999102,0.998877,0.999551,0.999102,0.999237,0.000269
10,37.687231,2.166094,0.493081,0.075351,None,150,"{'max_depth': None, 'n_estimators': 150}",0.979372,0.973944,0.974843,...,0.973954,0.003738,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
11,50.550413,6.744796,0.399162,0.060151,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977578,0.977538,0.973944,...,0.973594,0.003674,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,64.230061,1.984937,0.664223,0.161551,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.977538,0.973944,...,0.973415,0.003844,4,0.999326,0.999326,0.999102,0.999326,0.998877,0.999192,0.000180
3,7.453869,0.384169,0.804946,0.913897,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.978475,0.974843,0.976640,...,0.972517,0.005792,5,0.991015,0.992366,0.995510,0.994612,0.990121,0.992725,0.002056


In [16]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label']) # count_feat dataset
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are acc

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
7,35.497479,1.789654,0.502855,0.035065,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977578,0.973944,0.972147,...,0.973056,0.003226,1,0.998877,0.998653,0.999102,0.999551,0.998653,0.998967,0.000336
8,65.550581,1.696144,0.676391,0.129027,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.974843,0.973944,...,0.972517,0.003541,2,0.999326,0.998877,0.998877,0.999326,0.998877,0.999057,0.000220
10,38.525191,1.467632,0.527389,0.050502,None,150,"{'max_depth': None, 'n_estimators': 150}",0.980269,0.971249,0.974843,...,0.971978,0.005171,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
11,49.002975,7.789390,0.394745,0.064095,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975785,0.973046,0.972147,...,0.971798,0.003255,4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
6,9.074350,4.926153,0.721212,0.469295,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.973094,0.973944,0.973046,...,0.971080,0.003290,5,0.998203,0.997081,0.997979,0.998204,0.997306,0.997755,0.000471
